# Compile results

In [ ]:
import os
import zipfile

from tqdm.notebook import tqdm


In [ ]:
import itertools
import json
import warnings

from numpy import arange
from numpy.random import default_rng


In [ ]:
from pNeuma_simulator.results import aggregate, percolate


In [8]:
warnings.filterwarnings("ignore")

In [9]:
path = "./output/"
os.makedirs(path, exist_ok=True)
# Combinatorial configurations
n_veh = 8
scale = 2
l_cars = scale * arange(1, n_veh, 1)
l_moto = scale * arange(0, n_veh - 1, 1)
permutations = list(itertools.product(l_cars, l_moto))
len(permutations)

49

In [10]:
# consider only this last fraction of the simulated data
keep = 1 / 3
# Pseudorandom number generator
seed = 1024
rng = default_rng(seed)
compile = True

In [ ]:
path = "./output/"
os.makedirs(path, exist_ok=True)
with open(f"{path}results_.json") as openfile:
    results = json.load(openfile)

In [12]:
n_cars = 12
n_moto = 6
verbose = True
items = []
with zipfile.ZipFile(f"{path}{(n_cars, n_moto)}.zip", "r") as ziph:
    # ziph is zipfile handle
    for filename in ziph.namelist():
        if filename.endswith(").jsonl"):
            # Opening JSONL file
            with ziph.open(filename, "r") as openfile:
                # Reading from JSONL file
                # https://stackoverflow.com/questions/2081836/
                for i, line in enumerate(tqdm(openfile)):
                    item = json.loads(line)
                    if isinstance(item[0], list):
                        items.append(item)
                if verbose:
                    print(openfile.name)

0it [00:00, ?it/s]

(12, 6).jsonl


In [13]:
if compile:
    # results = {}
    for permutation in [(n_cars, n_moto)]:  # permutations:
        # n_cars, n_moto = permutation
        results[str(permutation)] = {
            "VKT_cars": [],
            "VHT_cars": [],
            "VKT_moto": [],
            "VHT_moto": [],
        }
        # items = loader(permutation, path)
        accidents = 0
        for item in items:
            if isinstance(item[0], list):
                (
                    VKT_cars,
                    VHT_cars,
                    VKT_moto,
                    VHT_moto,
                ) = aggregate(item[0], n_cars, n_moto)
                results[str(permutation)]["VKT_cars"].append(VKT_cars)
                results[str(permutation)]["VHT_cars"].append(VHT_cars)
                results[str(permutation)]["VKT_moto"].append(VKT_moto)
                results[str(permutation)]["VHT_moto"].append(VHT_moto)
            else:
                accidents += 1
        error = accidents / len(items)
        results[str(permutation)]["error"] = error
        if n_cars > 2 and n_moto > 2:
            percolation = percolate(items, n_cars, n_moto, rng, start=5)
        else:
            percolation = None
        results[str(permutation)]["percolation"] = percolation
        items = None

In [14]:
write = compile
if write:
    with open(f"{path}results_.json", "w") as outfile:
        json.dump(results, outfile)